In [ ]:
%matplotlib widget
import hyperspy.api as hs
import numpy as np

### Load the raw simulated data

In [ ]:
s = hs.load('*.h5', stack=True, new_axis_name='occupancy', reader='emd')

In [ ]:
s

In [ ]:
im = s[1]
im.axes_manager['occupancy'].scale = 0.1

In [ ]:
im.axes_manager

### Fix calibration dataset
This is a known bug in prismatic 1.2.1:
- signal axis units are A instead of nm
- scattering angle is in rad instead of mrad

In [ ]:
for axis in im.axes_manager.signal_axes:
    axis.units = 'Å'
im.axes_manager['bin_outer_angle'].scale *= 1000

In [ ]:
im.plot(navigator='spectrum')

### Visualise simulated images

In [ ]:
roi = hs.roi.SpanROI(50, 100)
sliced_signal = roi.interactive(im)

In [ ]:
axis = 0
out = sliced_signal.sum(axis=axis)
hs.interactive(sliced_signal.sum,
               axis=axis,
               recompute_out_event=None,
               out=out)
out.plot(navigator='slider')

### Get HAADF images

In [ ]:
HAADF_im = roi(im).sum(0)

In [ ]:
HAADF_im.plot()

### Apply a Gaussian filter to model the probe size

In [ ]:
from scipy.ndimage import gaussian_filter

FWHM_pixel = 0.8 / im.axes_manager[-1].scale
sigma_pixel = np.round(FWHM_pixel / 2.344, 2)

im_gaussian_filtered = im.map(gaussian_filter, inplace=False, sigma=sigma_pixel)

In [ ]:
im_gaussian_filtered.plot()

### Get HAADF images including probe size

In [ ]:
roi.left = 60
roi.right = 120

In [ ]:
images = roi(im_gaussian_filtered).sum(0)

Same as above but this syntax is more robust and explicit, because the axis sliced by the ROI is specified

In [ ]:
images = im_gaussian_filtered.inav[slice(*roi), :].sum(0)

In [ ]:
images.plot()

### Make a figure to compare the simulated images

In [ ]:
labels = images.axes_manager[0].axis
labels = [f"Ge: {label:.2g}" for label in labels]

In [ ]:
hs.plot.plot_images(images, axes_decor='off', label=labels, colorbar='single', suptitle='Ge occupancy')